Species Comparison View and Conservation Priority Index (CPI)

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
df=pd.read_csv("Wildlife_Dataset_Top6.csv")

In [3]:
df_feature=pd.read_csv("Feature_Matrix.csv")

In [4]:
df_output=pd.read_csv("Final_Output.csv")

Create Species-Level Summary Table

In [5]:
# % Live Trade per Species
live_trade_species = (
    df.groupby("Taxon")["is_live"]
    .mean()
    .reset_index(name="live_trade_ratio")
)

In [6]:
# Mean Appendix Severity per Species
appendix_species = (
    df_feature.groupby("Taxon")["appendix_risk"]
    .mean()
    .reset_index(name="mean_appendix_risk")
)

In [7]:
# Top Exporter Country per Species
top_exporter_species = (
    df.groupby("Taxon")["Exporter"]
    .agg(lambda x: x.value_counts().idxmax())
    .reset_index(name="top_exporter")
)

In [8]:
# Mean Risk Score per Species (from model output)
risk_species = (
    df_output.groupby("Taxon")["poaching_risk_score"]
    .mean()
    .reset_index(name="poaching_risk_score")
)

In [9]:
species_summary = (
    live_trade_species
        .merge(appendix_species, on="Taxon")
        .merge(top_exporter_species, on="Taxon")
        .merge(risk_species, on="Taxon")
        .sort_values("poaching_risk_score", ascending=False)
)

In [11]:
# Computing CPI
epsilon = 0.05
species_summary["CPI"] = (
    (species_summary["poaching_risk_score"] + epsilon)
    * species_summary["mean_appendix_risk"]
    * (1 + 0.5 * species_summary["live_trade_ratio"])
)

In [12]:
# Sort by CPI
output_df = species_summary.sort_values(
    "CPI",
    ascending=False
)

In [13]:
output_df

,Taxon,live_trade_ratio,mean_appendix_risk,top_exporter,poaching_risk_score,CPI
1,Crocodylus Niloticus,0.005446,2.050203,Italy,1.000000,2.158575
4,Python Reticulatus,0.016898,2.000000,Italy,0.982608,2.082666
2,Crocodylus Porosus,0.002137,2.144619,France,0.858284,1.950004
0,Alligator Mississippiensis,0.009052,2.000000,Italy,0.803649,1.715025
3,Python Bivittatus,0.031453,2.000000,Italy,0.724669,1.573703
5,Varanus Salvator,0.032292,2.000000,Italy,0.642258,1.406869


Downloadable CSV File

In [14]:
output_df.to_csv("Species_Summary.csv", index=False)
from google.colab import files
files.download("Species_Summary.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>